In [15]:
import pathlib as pl
from typing import Optional, List

import folium as fl
import branca as bc
import matplotlib as mpl
from matplotlib import cm
import geopandas as gp
import numpy as np


# Configure

In [9]:
DATA_DIR = pl.Path("../data")

%ls {DATA_DIR}

statistical-area-1-2019-clipped-generalised.gpkg


# Make helpers

In [73]:
WGS84 = "epsg:4326"

def add_categorical_legend(
    folium_map: fl.Map,
    title: str,
    colors: List[str],
    labels: List[str],
) -> fl.Map:
    """
    Given a Folium map, add to it a categorical legend with the given title, colors, and corresponding labels.
    The given colors and labels will be listed in the legend from top to bottom.
    Return the resulting map.
    
    Based on `this example <http://nbviewer.jupyter.org/gist/talbertc-usgs/18f8901fc98f109f2b71156cf3ac81cd>`_.
    """
    # Error check
    if len(colors) != len(labels):
        raise ValueError("colors and labels must have the same length.")

    color_by_label = dict(zip(labels, colors))

    # Make legend HTML
    template = f"""
    {{% macro html(this, kwargs) %}}

    <!doctype html>
    <html lang="en">
    <head>
      <meta charset="utf-8">
      <meta name="viewport" content="width=device-width, initial-scale=1">
    </head>
    <body>
    <div id='maplegend' class='maplegend'>
      <div class='legend-title'>{title}</div>
      <div class='legend-scale'>
        <ul class='legend-labels'>
    """

    for label, color in color_by_label.items():
        template += f"<li><span style='background:{color}'></span>{label}</li>"

    template += """
        </ul>
      </div>
    </div>

    </body>
    </html>

    <style type='text/css'>
      .maplegend {
        position: absolute;
        z-index:9999;
        background-color: rgba(255, 255, 255, 1);
        border-radius: 5px;
        border: 2px solid #bbb;
        padding: 10px;
        font-size:12px;
        right: 10px;
        bottom: 20px;
      }
      .maplegend .legend-title {
        text-align: left;
        margin-bottom: 5px;
        font-weight: bold;
        font-size: 90%;
        }
      .maplegend .legend-scale ul {
        margin: 0;
        margin-bottom: 5px;
        padding: 0;
        float: left;
        list-style: none;
        }
      .maplegend .legend-scale ul li {
        font-size: 80%;
        list-style: none;
        margin-left: 0;
        line-height: 18px;
        margin-bottom: 2px;
        }
      .maplegend ul.legend-labels li span {
        display: block;
        float: left;
        height: 16px;
        width: 30px;
        margin-right: 5px;
        margin-left: 0;
        border: 0px solid #ccc;
        }
      .maplegend .legend-source {
        font-size: 80%;
        color: #777;
        clear: both;
        }
      .maplegend a {
        color: #777;
        }
    </style>
    {% endmacro %}
    """

    macro = bc.element.MacroElement()
    macro._template = bc.element.Template(template)
    folium_map.get_root().add_child(macro)

    return folium_map




# Create a map with a categorical legend

In [87]:
# Load geodata features (polygons)
path = DATA_DIR/"statistical-area-1-2019-clipped-generalised.gpkg"
g = gp.read_file(path).to_crs(WGS84)

# Use colors from https://colorbrewer2.org/#type=qualitative&scheme=Dark2&n=5
colors = ['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e']
categories = ["A", "B", "C", "D", "E"]

# Randomly assign categories to features
n = len(categories)
g["category"] = np.random.choice(categories, g.shape[0])
display(g.head())

# Color by category
color_dict = { 
    cat: colors[categories.index(cat)]
    for cat in g.category.values
}

# Initialize map
my_map = fl.Map(tiles=None)
fl.TileLayer(tiles="cartodbpositron", attr="", name="basemap").add_to(my_map)

bounds = g.total_bounds + 0.0001
bounds = [(bounds[1], bounds[0]), (bounds[3], bounds[2])]  # rearrange for Folium
my_map.fit_bounds(bounds)
    
# Add map features as GeoJSON
fl.GeoJson(
    g,
    name="SA1s",
    style_function=lambda feature: {
        "fillColor": color_dict[feature["properties"]["category"]],
        "color": "black",
        "weight": 1,
        "fillOpacity": 1,
    },
    highlight_function=lambda x: {"weight": 3},
    tooltip=fl.features.GeoJsonTooltip(
        fields=["SA12019_V1_00", "category"],
        aliases=["SA1 code", "category"],
        sticky=True,
        opacity=0.9,
        direction="bottom",
    ),
).add_to(my_map)

# Add map layer control
fl.LayerControl(collapsed=True).add_to(my_map)

# Add map legend 
my_map = add_categorical_legend(my_map, "Category", colors=colors, labels=categories)

# Add map title
my_map.get_root().html.add_child(fl.Element(f"<h1>Auckland Downtown SA1s</h1>"))

my_map

,SA12019_V1_00,LANDWATER,LANDWATER_NAME,LAND_AREA_SQ_KM,AREA_SQ_KM,Shape_Length,geometry,category
0,7004933,12,Mainland,0.032764,0.032764,735.543818,"MULTIPOLYGON (((174.74447 -36.86063, 174.74459...",C
1,7004934,12,Mainland,0.033160,0.033160,731.559633,"MULTIPOLYGON (((174.74483 -36.86247, 174.74447...",E
2,7004936,12,Mainland,0.025346,0.025346,698.390053,"MULTIPOLYGON (((174.74447 -36.86358, 174.74484...",A
3,7004939,12,Mainland,0.039015,0.039015,1175.045696,"MULTIPOLYGON (((174.74447 -36.86720, 174.74458...",D
4,7004941,12,Mainland,0.039592,0.039592,1162.596435,"MULTIPOLYGON (((174.74447 -36.86746, 174.74458...",A


[174.74456813 -36.86835675 174.78284871 -36.83509965]
